In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC50_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_LRup_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_LRup_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4] + "_LRup"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC50(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df[df.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = 60
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC50(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_15384\2625783892.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
Data Loading Time: 132.91 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 50)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX5
            dataY = dataY5
            lr = 0.0008
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            lr = 0.0016
        else:
            #5s
            dataX = dataX1
            dataY = dataY1
            lr = 0.0032

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.7535128593444824 Train_acc: 2.312500%
Train Epoch: 2/60 Train_Loss: 3.803389072418213 Train_acc: 4.062500%
Train Epoch: 3/60 Train_Loss: 3.5918240547180176 Train_acc: 5.250000%
Train Epoch: 4/60 Train_Loss: 3.589045286178589 Train_acc: 6.625000%
Train Epoch: 5/60 Train_Loss: 3.4237875938415527 Train_acc: 6.125000%
Train Epoch: 6/60 Train_Loss: 3.476003646850586 Train_acc: 6.500000%
Train Epoch: 7/60 Train_Loss: 3.5151519775390625 Train_acc: 7.687500%
Train Epoch: 8/60 Train_Loss: 3.3635642528533936 Train_acc: 8.062500%
Train Epoch: 9/60 Train_Loss: 3.377537250518799 Train_acc: 8.687500%
Train Epoch: 10/60 Train_Loss: 3.3709840774536133 Train_acc: 10.187500%
Train Epoch: 11/60 Train_Loss: 3.2827894687652

Train Epoch: 47/60 Train_Loss: 0.8645808100700378 Train_acc: 74.979172%
Train Epoch: 48/60 Train_Loss: 0.7628580331802368 Train_acc: 74.854172%
Train Epoch: 49/60 Train_Loss: 0.7568997144699097 Train_acc: 75.250000%
Train Epoch: 50/60 Train_Loss: 0.6844460368156433 Train_acc: 75.291672%
Train Epoch: 51/60 Train_Loss: 0.8070892691612244 Train_acc: 76.104172%
Train Epoch: 52/60 Train_Loss: 0.7250155210494995 Train_acc: 76.083336%
Train Epoch: 53/60 Train_Loss: 0.7262208461761475 Train_acc: 76.458336%
Train Epoch: 54/60 Train_Loss: 0.8039213418960571 Train_acc: 75.729172%
Train Epoch: 55/60 Train_Loss: 0.7928429245948792 Train_acc: 75.854172%
Train Epoch: 56/60 Train_Loss: 0.8727114200592041 Train_acc: 76.562500%
Train Epoch: 57/60 Train_Loss: 0.5770648121833801 Train_acc: 76.666672%
Train Epoch: 58/60 Train_Loss: 0.939180314540863 Train_acc: 76.604172%
Train Epoch: 59/60 Train_Loss: 0.6510453820228577 Train_acc: 74.958336%
Train Epoch: 60/60 Train_Loss: 0.6038957238197327 Train_acc: 76.8

Train Epoch: 31/60 Train_Loss: 2.072000503540039 Train_acc: 34.937500%
Train Epoch: 32/60 Train_Loss: 1.9516271352767944 Train_acc: 35.687500%
Train Epoch: 33/60 Train_Loss: 1.951608657836914 Train_acc: 35.937500%
Train Epoch: 34/60 Train_Loss: 2.1367361545562744 Train_acc: 36.812500%
Train Epoch: 35/60 Train_Loss: 2.2262814044952393 Train_acc: 36.937500%
Train Epoch: 36/60 Train_Loss: 1.963257074356079 Train_acc: 36.687500%
Train Epoch: 37/60 Train_Loss: 1.927980899810791 Train_acc: 36.250000%
Train Epoch: 38/60 Train_Loss: 2.3200435638427734 Train_acc: 36.687500%
Train Epoch: 39/60 Train_Loss: 2.1872737407684326 Train_acc: 37.187500%
Train Epoch: 40/60 Train_Loss: 2.058093547821045 Train_acc: 36.625000%
Train Epoch: 41/60 Train_Loss: 2.0133891105651855 Train_acc: 37.250000%
Train Epoch: 42/60 Train_Loss: 2.009117364883423 Train_acc: 38.500000%
Train Epoch: 43/60 Train_Loss: 2.063098192214966 Train_acc: 37.625000%
Train Epoch: 44/60 Train_Loss: 2.055170774459839 Train_acc: 39.062500%


Train Epoch: 15/60 Train_Loss: 1.661026954650879 Train_acc: 50.437504%
Train Epoch: 16/60 Train_Loss: 1.0999267101287842 Train_acc: 58.787502%
Train Epoch: 17/60 Train_Loss: 1.0514650344848633 Train_acc: 62.375004%
Train Epoch: 18/60 Train_Loss: 1.1176648139953613 Train_acc: 61.262505%
Train Epoch: 19/60 Train_Loss: 1.28680419921875 Train_acc: 63.200005%
Train Epoch: 20/60 Train_Loss: 1.6364247798919678 Train_acc: 61.375004%
Train Epoch: 21/60 Train_Loss: 1.0396857261657715 Train_acc: 63.087502%
Train Epoch: 22/60 Train_Loss: 1.3491175174713135 Train_acc: 64.450005%
Train Epoch: 23/60 Train_Loss: 1.132694125175476 Train_acc: 63.200005%
Train Epoch: 24/60 Train_Loss: 0.8799998760223389 Train_acc: 63.475002%
Train Epoch: 25/60 Train_Loss: 1.1537445783615112 Train_acc: 65.137505%
Train Epoch: 26/60 Train_Loss: 0.7791480422019958 Train_acc: 65.375000%
Train Epoch: 27/60 Train_Loss: 1.045877456665039 Train_acc: 65.550003%
Train Epoch: 28/60 Train_Loss: 1.1577744483947754 Train_acc: 63.62500

Train Epoch: 1/60 Train_Loss: 2.448760747909546 Train_acc: 26.604168%
Train Epoch: 2/60 Train_Loss: 2.3965795040130615 Train_acc: 28.000000%
Train Epoch: 3/60 Train_Loss: 2.3299624919891357 Train_acc: 31.145834%
Train Epoch: 4/60 Train_Loss: 2.052767276763916 Train_acc: 32.854168%
Train Epoch: 5/60 Train_Loss: 2.121936798095703 Train_acc: 35.479168%
Train Epoch: 6/60 Train_Loss: 2.0339841842651367 Train_acc: 36.479168%
Train Epoch: 7/60 Train_Loss: 2.18891978263855 Train_acc: 37.625000%
Train Epoch: 8/60 Train_Loss: 1.964306354522705 Train_acc: 38.833336%
Train Epoch: 9/60 Train_Loss: 1.9376471042633057 Train_acc: 41.083336%
Train Epoch: 10/60 Train_Loss: 1.956015706062317 Train_acc: 42.375000%
Train Epoch: 11/60 Train_Loss: 1.5288057327270508 Train_acc: 43.937500%
Train Epoch: 12/60 Train_Loss: 1.5732204914093018 Train_acc: 47.083336%
Train Epoch: 13/60 Train_Loss: 1.528299331665039 Train_acc: 47.166668%
Train Epoch: 14/60 Train_Loss: 1.3732597827911377 Train_acc: 50.812500%
Train Epo

Train Epoch: 50/60 Train_Loss: 0.8684073686599731 Train_acc: 81.537506%
Train Epoch: 51/60 Train_Loss: 0.627107560634613 Train_acc: 81.500008%
Train Epoch: 52/60 Train_Loss: 0.5979394912719727 Train_acc: 81.812508%
Train Epoch: 53/60 Train_Loss: 0.8349393606185913 Train_acc: 81.337502%
Train Epoch: 54/60 Train_Loss: 0.5115816593170166 Train_acc: 81.150002%
Train Epoch: 55/60 Train_Loss: 0.47495415806770325 Train_acc: 80.912506%
Train Epoch: 56/60 Train_Loss: 0.4866790473461151 Train_acc: 81.937508%
Train Epoch: 57/60 Train_Loss: 0.6781920790672302 Train_acc: 82.387505%
Train Epoch: 58/60 Train_Loss: 0.8239530324935913 Train_acc: 82.237503%
Train Epoch: 59/60 Train_Loss: 0.8244473338127136 Train_acc: 82.500008%
Train Epoch: 60/60 Train_Loss: 0.6532323360443115 Train_acc: 82.662506%
Test accuracy of the model: 38.600000%
Test accuracy of the model: 42.666667%
Test accuracy of the model: 42.250000%
Training Time: 52.33 seconds
可用的 GPU 数量: 1
------------------------------------------------

Train Epoch: 34/60 Train_Loss: 1.0783679485321045 Train_acc: 69.125000%
Train Epoch: 35/60 Train_Loss: 1.0311497449874878 Train_acc: 68.354172%
Train Epoch: 36/60 Train_Loss: 0.9105732440948486 Train_acc: 70.145836%
Train Epoch: 37/60 Train_Loss: 0.9689406156539917 Train_acc: 70.375000%
Train Epoch: 38/60 Train_Loss: 0.9940786957740784 Train_acc: 70.208336%
Train Epoch: 39/60 Train_Loss: 0.7218312621116638 Train_acc: 70.583336%
Train Epoch: 40/60 Train_Loss: 0.8173178434371948 Train_acc: 70.395836%
Train Epoch: 41/60 Train_Loss: 0.6657582521438599 Train_acc: 71.208336%
Train Epoch: 42/60 Train_Loss: 0.8026185035705566 Train_acc: 71.562500%
Train Epoch: 43/60 Train_Loss: 1.0078569650650024 Train_acc: 71.979172%
Train Epoch: 44/60 Train_Loss: 0.809985339641571 Train_acc: 72.166672%
Train Epoch: 45/60 Train_Loss: 0.7782496213912964 Train_acc: 72.270836%
Train Epoch: 46/60 Train_Loss: 0.6683326363563538 Train_acc: 74.687500%
Train Epoch: 47/60 Train_Loss: 0.6896104216575623 Train_acc: 74.8

Train Epoch: 18/60 Train_Loss: 2.4450197219848633 Train_acc: 30.437500%
Train Epoch: 19/60 Train_Loss: 2.4554553031921387 Train_acc: 30.500000%
Train Epoch: 20/60 Train_Loss: 2.5499660968780518 Train_acc: 31.500000%
Train Epoch: 21/60 Train_Loss: 2.1071829795837402 Train_acc: 30.937500%
Train Epoch: 22/60 Train_Loss: 2.186937093734741 Train_acc: 32.625000%
Train Epoch: 23/60 Train_Loss: 2.2454733848571777 Train_acc: 32.875000%
Train Epoch: 24/60 Train_Loss: 2.1585333347320557 Train_acc: 33.812500%
Train Epoch: 25/60 Train_Loss: 2.0768139362335205 Train_acc: 35.125000%
Train Epoch: 26/60 Train_Loss: 2.201995372772217 Train_acc: 34.062500%
Train Epoch: 27/60 Train_Loss: 2.2109439373016357 Train_acc: 35.937500%
Train Epoch: 28/60 Train_Loss: 2.2226901054382324 Train_acc: 36.812500%
Train Epoch: 29/60 Train_Loss: 2.3912041187286377 Train_acc: 38.437500%
Train Epoch: 30/60 Train_Loss: 2.0167202949523926 Train_acc: 37.312500%
Train Epoch: 31/60 Train_Loss: 2.1249845027923584 Train_acc: 39.62

Train Epoch: 2/60 Train_Loss: 1.550065279006958 Train_acc: 41.437504%
Train Epoch: 3/60 Train_Loss: 2.339743137359619 Train_acc: 42.825001%
Train Epoch: 4/60 Train_Loss: 2.3474576473236084 Train_acc: 43.775002%
Train Epoch: 5/60 Train_Loss: 1.7649704217910767 Train_acc: 44.675003%
Train Epoch: 6/60 Train_Loss: 1.8647922277450562 Train_acc: 47.037502%
Train Epoch: 7/60 Train_Loss: 1.71664297580719 Train_acc: 47.200001%
Train Epoch: 8/60 Train_Loss: 1.553508996963501 Train_acc: 48.675003%
Train Epoch: 9/60 Train_Loss: 1.7351248264312744 Train_acc: 49.050003%
Train Epoch: 10/60 Train_Loss: 2.0697567462921143 Train_acc: 49.987503%
Train Epoch: 11/60 Train_Loss: 1.3577980995178223 Train_acc: 49.887501%
Train Epoch: 12/60 Train_Loss: 1.7623448371887207 Train_acc: 49.175003%
Train Epoch: 13/60 Train_Loss: 2.363793134689331 Train_acc: 51.450001%
Train Epoch: 14/60 Train_Loss: 1.7334001064300537 Train_acc: 51.500004%
Train Epoch: 15/60 Train_Loss: 1.605507254600525 Train_acc: 53.537502%
Train E

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()